In [1]:
pip install -U vnstock

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=89e35d6e6268ba4aaae6f5b5c196bbd076323b8c36cd62a540c5b70585557e42
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=dbf9e6f035a5450a373103d08a7992d2b06aad31409451117d58b3a4f34f433d
  Stored in directory: /root/.cache/pip/wheels/46/f9/96/709295c836133071c12a300729fed4027757f889c01695feea
Successfully built keras-self-attention
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
from ta.momentum import RSIIndicator
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, RNN, concatenate
from tensorflow.keras.optimizers import RMSprop
from vnstock import *
from ta.volatility import BollingerBands
from tensorflow.keras.layers import Layer
import tensorflow as tf
from keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, Flatten, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier

from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import CCIIndicator
from sklearn.preprocessing import StandardScaler
from ta.volume import ChaikinMoneyFlowIndicator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import regularizers
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.saving import register_keras_serializable
import matplotlib.pyplot as plt
# from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from vnstock import *
import pandas as pd
import os
from collections import Counter
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
from sklearn.metrics import accuracy_score # Import accuracy
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
import optuna
import joblib
from sklearn.metrics import (
    f1_score, accuracy_score, classification_report, roc_auc_score,
    roc_curve, auc
)
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import TimeSeriesSplit

2025-07-24 11:14:44.695969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753355684.937078      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753355685.005613      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [9]:
def dataframe_to_x_y(df, feature, target):
  X_train, y_train = [], []
  X_val, y_val = [], []
  n = len(df)
  split_index = int(n * 0.8)
  X_train = df[feature][:split_index]
  y_train = df[target][:split_index]
  X_val = df[feature][split_index:]
  y_val = df[target][split_index:]
  return X_train, y_train, X_val, y_val

In [10]:

sentiment_columns = [
        'Reputation', 'Company Communication', 'Appointment', 'Financial', 'Regulatory',
        'Sales', 'M&A', 'Legal', 'Dividend Policy', 'Risks', 'Rumors', 'Strategy',
        'Options', 'IPO', 'Signal', 'Coverage', 'Fundamentals', 'Insider Activity',
        'Price Action', 'Buyside', 'Technical Analysis', 'Trade', 'Central Banks',
        'Currency', 'Conditions', 'Market', 'Volatility', 'Investor Sentiment', 'Retail Investor Behavior',
        'Speculation', 'Domestic Institutional Behavior', 'Foreign Institutional Behavior', 'Black Swan Event',
    ]

sentiment_p1d_columns = [
        'reputation_p1d', 'company_communication_p1d', 'appointment_p1d', 'financial_p1d', 'regulatory_p1d',
        'sales_p1d', 'm&a_p1d', 'legal_p1d', 'dividend_policy_p1d', 'risks_p1d', 'rumors_p1d', 'strategy_p1d',
        'options_p1d', 'ipo_p1d', 'signal_p1d', 'coverage_p1d', 'fundamentals_p1d', 'insider_activity_p1d',
        'price_action_p1d', 'buyside_p1d', 'technical_analysis_p1d', 'trade_p1d', 'central_banks_p1d',
        'currency_p1d', 'conditions_p1d', 'market_p1d', 'volatility_p1d', 'investor_sentiment_p1d', 'retail_investor_behavior_p1d',
        'speculation_p1d', 'domestic_institutional_behavior_p1d', 'foreign_institutional_behavior_p1d', 'black_swan_event_p1d',
    ]
meta_cols = ['num_sa_news', 'mean_sentiment_norm', 'max_sentiment_norm', 'std_sentiment_score', 'sum_positive_sa', 'sum_negative_sa', 'time_distance_from_sa_to_close_minutes']
meta_p1d_cols = ['num_sa_news_p1d', 'mean_sentiment_norm_p1d', 'max_sentiment_norm_p1d', 'std_sentiment_score_p1d', 'sum_positive_sa_p1d', 'sum_negative_sa_p1d', 'time_distance_from_sa_to_close_minutes_p1d']

sentiment_feature_selected =  sentiment_columns + sentiment_p1d_columns + meta_cols + meta_p1d_cols
ta_features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','roc_1','roc_9','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','rsi_base_ma','rsi_rsi_base_ma_ratio','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','hl_ratio', 'co_ratio', 'price_range', 'sma_ratio_20_50', 'ema_ratio_12_26', 'bb_width', 'bb_position', 'rsi_overbought', 'rsi_oversold', 'rsi_neutral', 'macd_bullish', 'momentum_5', 'momentum_10','log_return','volatility_5d','volatility_10d','volatility_20d','volatility_30d','mean_log_return_5d','mean_log_return_10d','mean_log_return_20d','mean_log_return_30d','sharpe_like_5d','sharpe_like_10d','sharpe_like_20d','sharpe_like_30d','up_streak','pos_log_return_ratio_20d','z_score_5d','z_score_10d','z_score_20d','z_score_30d','annual_return','daily_return','sharpe_ratio',
               'rsi_vn30','rsi_base_ma_vn30','rsi_rsi_base_ma_ratio_vn30','volume_ma_vn30','volume_to_volume_ma_ratio_vn30','bb_bbm_vn30','bb_bbh_vn30','bb_bbl_vn30','bb_bbp_vn30','bb_bbh_bb_bbl_ratio_vn30','roc_1_vn30', 'roc_5_vn30', 'roc_9_vn30','%K_vn30','%R_vn30','cci_vn30','obv_vn30','ema_12_vn30','ema_26_vn30','sma_20_vn30','sma_50_vn30', 'hl_ratio_vn30', 'co_ratio_vn30', 'price_range_vn30', 'sma_ratio_20_50_vn30', 'ema_ratio_12_26_vn30', 'bb_width_vn30', 'bb_position_vn30', 'rsi_overbought_vn30', 'rsi_oversold_vn30', 'rsi_neutral_vn30', 'momentum_5_vn30', 'momentum_10_vn30', 'log_return_vn30','volatility_5d_vn30','volatility_10d_vn30','volatility_20d_vn30','volatility_30d_vn30','mean_log_return_5d_vn30','mean_log_return_10d_vn30','mean_log_return_20d_vn30','mean_log_return_30d_vn30','sharpe_like_5d_vn30','sharpe_like_10d_vn30','sharpe_like_20d_vn30','sharpe_like_30d_vn30','up_streak_vn30','pos_log_return_ratio_20d_vn30','z_score_5d_vn30','z_score_10d_vn30','z_score_20d_vn30','z_score_30d_vn30','annual_return_vn30','daily_return_vn30','sharpe_ratio_vn30',
               'rsi_vni','rsi_base_ma_vni','rsi_rsi_base_ma_ratio_vni','volume_ma_vni','volume_to_volume_ma_ratio_vni','bb_bbm_vni','bb_bbh_vni','bb_bbl_vni','bb_bbp_vni','bb_bbh_bb_bbl_ratio_vni','roc_1_vni', 'roc_5_vni', 'roc_9_vni','%K_vni','%R_vni','cci_vni','obv_vni','ema_12_vni','ema_26_vni','sma_20_vni','sma_50_vni', 'hl_ratio_vni', 'co_ratio_vni', 'price_range_vni', 'sma_ratio_20_50_vni', 'ema_ratio_12_26_vni', 'bb_width_vni', 'bb_position_vni', 'rsi_overbought_vni', 'rsi_oversold_vni', 'rsi_neutral_vni', 'momentum_5_vni', 'momentum_10_vni','log_return_vni','volatility_5d_vni','volatility_10d_vni','volatility_20d_vni','volatility_30d_vni','mean_log_return_5d_vni','mean_log_return_10d_vni','mean_log_return_20d_vni','mean_log_return_30d_vni','sharpe_like_5d_vni','sharpe_like_10d_vni','sharpe_like_20d_vni','sharpe_like_30d_vni','up_streak_vni','pos_log_return_ratio_20d_vni','z_score_5d_vni','z_score_10d_vni','z_score_20d_vni','z_score_30d_vni','annual_return_vni','daily_return_vni','sharpe_ratio_vni']
fa_features = ['p/b_previous_quarter', 'p/b_change_rate','p/b_change_rate_flag','p/e_previous_quarter','p/e_change_rate','p/e_change_rate_flag','p/s_previous_quarter','p/s_change_rate','p/s_change_rate_flag','p/cash_flow_previous_quarter','p/cash_flow_change_rate','p/cash_flow_change_rate_flag','eps_previous_quarter','eps_change_rate', 'eps_change_rate_flag','bvps_previous_quarter','bvps_change_rate', 'bvps_change_rate_flag','roe_previous_quarter','roe_change_rate','roe_change_rate_flag','roa_previous_quarter','roa_change_rate','roa_change_rate_flag','coefficient_p/b','coefficient_p/e','coefficient_p/s','coefficient_p/cash_flow','coefficient_eps','coefficient_bvps','coefficient_roe','coefficient_roa','distance_to_nearest_quarter']
ta_fa_feature_selected = ta_features + fa_features
# features = ta_fa_feature_selected + sentiment_feature_selected
features = ta_features
target = 'target'
scaler = StandardScaler()

In [11]:
def load_data_for_analysis(tickers, folder_path):

    df_all = []

    for ticker in tickers:
        file_path = os.path.join(folder_path, f"{ticker}.xlsx")
        if os.path.exists(file_path):
            df = pd.read_excel(file_path)
            df["ticker"] = ticker  # thêm cột ticker
            df_all.append(df)
        else:
            print(f"⚠️ Không tìm thấy: {file_path}")

    # Gộp lại thành 1 DataFrame
    df_global = pd.concat(df_all, ignore_index=True)
    print(f"✅ Đã load xong {len(df_global)} dòng dữ liệu từ {len(df_all)} cổ phiếu.")
    return df_global

In [12]:
tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG',
                    'LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB',
                    'TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
folder_path_train = "/kaggle/input/vn30-dataset/v6 - Copy/stock_trend/train_data" 
folder_path_test = "/kaggle/input/vn30-dataset/v6 - Copy/stock_trend/test_data" 
df_global_train = load_data_for_analysis(tickers, folder_path_train)
df_global_test = load_data_for_analysis(tickers, folder_path_test)


✅ Đã load xong 35865 dòng dữ liệu từ 30 cổ phiếu.
✅ Đã load xong 7380 dòng dữ liệu từ 30 cổ phiếu.


In [13]:
# Danh sách các mô hình
models = {
    "Random Forest": RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced'),
    "XGBoost": XGBClassifier(n_jobs=-1, random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
    "LightGBM": LGBMClassifier(n_jobs=-1, random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
}

In [14]:
X_train_no_sa = df_global_train[ta_features]
X_test_no_sa = df_global_test[ta_features]
y_train = df_global_train[target]
y_test = df_global_test[target]
scaler = StandardScaler()
X_train_no_sa_scaled = scaler.fit_transform(X_train_no_sa)
X_test_no_sa_scaled = scaler.transform(X_test_no_sa) 

In [ ]:
# ==============================================Tuning with Optuna + TimeSeriesSplit validate (AUC macro) ===============================================

Divide 5 fold to cross validation though tuning parameters

In [15]:
def get_grouped_time_series_folds(df, ticker_col='ticker', n_splits=5):
    tickers = df[ticker_col].unique()
    fold_indices = [[] for _ in range(n_splits)]  # List chứa n_splits fold tổng
    for ticker in tickers:
        idx = df[df[ticker_col] == ticker].index.values
        tscv = TimeSeriesSplit(n_splits=n_splits)
        for fold_num, (train_idx, val_idx) in enumerate(tscv.split(idx)):
            # Gộp index thực tế vào fold tổng tương ứng
            fold_indices[fold_num].append((idx[train_idx], idx[val_idx]))
    # Gộp index của từng fold tổng lại thành mảng train/val duy nhất
    grouped_folds = []
    for fold in fold_indices:
        train_idx = np.concatenate([item[0] for item in fold])
        val_idx = np.concatenate([item[1] for item in fold])
        grouped_folds.append((train_idx, val_idx))
    return grouped_folds

In [16]:
folds = get_grouped_time_series_folds(df_global_train, ticker_col='ticker', n_splits=5)

Tuning random forest for stock trend with ta + fa data

In [17]:
X_no_sa = X_train_no_sa_scaled  # hoặc X_train_with_sa_scaled nếu dùng cả SA
y = y_train

In [18]:
print("Tuning XGBClassifier with Optuna (AUC macro) ta features only")

def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
        "use_label_encoder": False,
        "eval_metric": "mlogloss",
        "random_state": 42,
        "tree_method": "hist",   # Sử dụng GPU
        "device": "cuda",
        "n_jobs": -1,
    }

    aucs = []
    for train_idx, val_idx in folds:
        X_train, X_val = X_no_sa[train_idx], X_no_sa[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        model = XGBClassifier(**params)
        model.fit(X_train, y_train_fold)
        # One-hot encode y_val_fold
        y_val_bin = label_binarize(y_val_fold, classes=[0,1,2])
        y_pred_proba = model.predict_proba(X_val)
        # Nếu predict_proba trả về list (multioutput), cần stack lại
        if isinstance(y_pred_proba, list):
            y_pred_proba = np.stack([p[:,1] for p in y_pred_proba], axis=1)
        auc_macro = roc_auc_score(y_val_bin, y_pred_proba, average="macro", multi_class="ovr")
        aucs.append(auc_macro)
    mean_auc = np.mean(aucs)
    print(f"Trial {trial.number}: AUC macro={mean_auc:.5f}, params={trial.params}")
    # Optuna sẽ maximize AUC macro
    return -mean_auc

study = optuna.create_study(direction="minimize")
study.optimize(objective_xgb, n_trials=50)

print("Best AUC macro (XGBoost):", -study.best_value)
print("Best params (XGBoost):", study.best_trial.params)

[I 2025-07-24 11:22:38,569] A new study created in memory with name: no-name-cb1b5874-d48e-40f0-b1e6-c338cd3d742f


Tuning XGBClassifier with Optuna (AUC macro) ta features only


[11:22:44] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

[I 2025-07-24 11:23:12,757] Trial 0 finished with value: -0.6285140174378022 and parameters: {'n_estimators': 569, 'max_depth': 7, 'learning_rate': 0.14648346632809353, 'subsample': 0.8383279763926275, 'colsample_bytree': 0.8546042445503335}. Best is trial 0 with value: -0.6285140174378022.


Trial 0: AUC macro=0.62851, params={'n_estimators': 569, 'max_depth': 7, 'learning_rate': 0.14648346632809353, 'subsample': 0.8383279763926275, 'colsample_bytree': 0.8546042445503335}


[I 2025-07-24 11:23:53,197] Trial 1 finished with value: -0.6315451950273566 and parameters: {'n_estimators': 374, 'max_depth': 10, 'learning_rate': 0.20279129175756427, 'subsample': 0.969923687702231, 'colsample_bytree': 0.7311599746930189}. Best is trial 1 with value: -0.6315451950273566.


...

Trial 47: AUC macro=0.64355, params={'n_estimators': 659, 'max_depth': 6, 'learning_rate': 0.03018470294653032, 'subsample': 0.7158299829708761, 'colsample_bytree': 0.7262969350092485}


[I 2025-07-24 12:56:05,872] Trial 48 finished with value: -0.64881055456694 and parameters: {'n_estimators': 816, 'max_depth': 14, 'learning_rate': 0.011651419272027837, 'subsample': 0.7560763935430721, 'colsample_bytree': 0.7617151556429856}. Best is trial 12 with value: -0.6512426387754884.


Trial 48: AUC macro=0.64881, params={'n_estimators': 816, 'max_depth': 14, 'learning_rate': 0.011651419272027837, 'subsample': 0.7560763935430721, 'colsample_bytree': 0.7617151556429856}


[I 2025-07-24 12:58:18,823] Trial 49 finished with value: -0.6429497742496498 and parameters: {'n_estimators': 872, 'max_depth': 11, 'learning_rate': 0.0387550296249187, 'subsample': 0.9459466455383579, 'colsample_bytree': 0.35199583312098026}. Best is trial 12 with value: -0.6512426387754884.


Trial 49: AUC macro=0.64295, params={'n_estimators': 872, 'max_depth': 11, 'learning_rate': 0.0387550296249187, 'subsample': 0.9459466455383579, 'colsample_bytree': 0.35199583312098026}
Best AUC macro (XGBoost): 0.6512426387754884
Best params (XGBoost): {'n_estimators': 794, 'max_depth': 12, 'learning_rate': 0.011137357517182185, 'subsample': 0.7133408583477188, 'colsample_bytree': 0.3011380525314675}
